# Gazprom Neft SmartOil Contest-2 (score -- 137.7980)

В данном kernel я привожу способ построения простейшей модели и её обучение на основе предоставленных данных. Особое внимание я уделил способу валидации. Так как валидация для временных рядов затруднена в связи с возможностью утечек данных. 

Я избрал подход, который основывается на следующей простой идее. Давайте будем делить нашу выборку на части по скважинам. Более формально запишем это так. Пусть есть набор скважин $\{X_i\}$, для каждой из которых есть последовательность измерений $T_i = {T_{ij}}$. Будем производть валидацию на различных подмножествах ${X_i}$. В задаче требуется предсказать значение переменной $Нефть, м3$ на 12 месяцев вперёд.

Покажем, как будем строить множества для обучения и валидации. Важной заметкой является то, что в качестве метрики выбрана $MAE$. В данной задаче она не слишком корректна. Но именно она определяет способ построения алгоритмов и обучающих множеств. Так как $MAE$ считается независимо для каждого измерения, предлагается строить независимые классификаторы для предсказания значения i-го месяца. Таким образом, для формирования ответа будем иметь 12 регрессоров.

Рассмотрим $X_i$. Будем делать предсказание на i-ый месяц вперёд по 12 предыдущим месяцам. Рассмотрим соответствующий набор $T_{i0}, T_{i1}, \cdots, T_{in_i}$. Тогда примерами, обучающей выборки для 1-го классификатора можно считать:

$\{T_{i0}, T_{i1}, T_{i2}, \cdots, T{i11}\} \rightarrow \{T_{i12}\}$

$\{T_{i1}, T_{i2}, T_{i3}, \cdots, T{i12}\} \rightarrow \{T_{i13}\}$

$\cdots$

$\{T_{ik}, T_{ik+1}, T_{ik+2}, \cdots, T{ik+11}\} \rightarrow \{T_{ik+12}\}$

Обучающими примерами для 2-го классификатора будут:

$\{T_{i0}, T_{i1}, T_{i2}, \cdots, T{i11}\} \rightarrow \{T_{i13}\}$

$\{T_{i1}, T_{i2}, T_{i3}, \cdots, T{i12}\} \rightarrow \{T_{i14}\}$

$\cdots$

$\{T_{ik}, T_{ik+1}, T_{ik+2}, \cdots, T{ik+11}\} \rightarrow \{T_{ik+13}\}$

и так далее. Если для какая-то последовательность оказывается не достаточно длинной, то можно заполнить все значения до первого измерения нулями.

Немного обсудим то, какие скважины брать. Ответ очень прост. Берём всё, что попадется под руку. Т.е. объединяем test_before и train. Эти множества не пересекаются.

In [1]:
import warnings

from xg

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import math
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
from tqdm import tqdm
import time

from tqdm import tqdm_notebook

pd.set_option('max_columns', 300)
pd.set_option('max_rows', 500)

/home/hedgehogues/anaconda3/envs/ozon/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv('task_3/train.csv', sep=',', encoding='cp1251')
test_before = pd.read_csv('task_3/test_before.csv', sep=',', encoding='cp1251')
order = pd.read_csv('order3.csv', sep=',', encoding='cp1251')

Выбросим все признаки, которые содержат $\text{NaN}$. Исключим из рассмотрения признаки *'Закачка, м3', 'Природный газ, м3', 'Газ из газовой шапки, м3', 'Конденсат, т'*, поскольку для целевых скважин они оказываются нулевыми.

In [3]:
train['Дата'] = train['Дата'].astype(pd.datetime)
test_before['Дата'] = test_before['Дата'].astype(pd.datetime)

train_cl = train.dropna(axis=1)
test_before_cl = test_before.dropna(axis=1)
train_and_valid = pd.concat([train_cl, test_before_cl], axis=0)
del train_and_valid['group']
train_and_valid = train_and_valid.sort_values(by=['Скважина', 'Дата']).groupby(by=['Скважина', 'Дата']).first()


train_and_valid['Вода, т'] = train_and_valid['Вода, т'].apply(lambda row: float(row.replace(',', '.')))
train_and_valid['Жидкость, т'] = train_and_valid['Жидкость, т'].apply(lambda row: float(row.replace(',', '.')))
train_and_valid['Конденсат, т'] = train_and_valid['Конденсат, т'].apply(lambda row: float(row.replace(',', '.')))
train_and_valid['Обводненность (масса), %'] = train_and_valid['Обводненность (масса), %'].apply(lambda row: float(row.replace(',', '.')))
train_and_valid['Нефть, м3'] = train_and_valid['Нефть, м3'].apply(lambda row: float(row.replace(',', '.')))
train_and_valid['Жидкость, м3'] = train_and_valid['Жидкость, м3'].apply(lambda row: float(row.replace(',', '.')))
train_and_valid['Добыча растворенного газа, м3'] = train_and_valid['Добыча растворенного газа, м3'].apply(lambda row: float(row.replace(',', '.')))
submit_wells = order['well']

submit_zeros_values = ['Закачка, м3', 'Природный газ, м3', 'Газ из газовой шапки, м3', 'Конденсат, т']
train_and_valid = train_and_valid.drop(submit_zeros_values, axis=1)

Ниже реализовано несколько функций. 

* train_valid_split -- разделяет выборку на train и valid по скважинам. Выдаёт train_x, train_y, valid_x, valid_y.  train_x -- содержит n сплитов (ровно также, как и train_y, valid_x, valid_y). i-ый сплит содержит 12 множеств, каждое из которых предназначается для i-го классификатора. min_lag -- минимальное допустимое число измерений для того, чтобы был добавлен пример в выборку. lag -- число измерений в обучающей выборке, predict_inds -- список месяцев вперёд, для которых нужно сделать предсказание.
* train_valid_split -- разделяет выборку на train и valid по скважинам. Выдаёт train_x, train_y, valid_x, valid_y.  train_x -- содержит n сплитов (ровно также, как и train_y, valid_x, valid_y). i-ый сплит содержит 12 множеств, каждое из которых предназначается для i-го классификатора. min_lag -- минимальное допустимое число измерений для того, чтобы был добавлен пример в выборку. lag -- число измерений в обучающей выборке, predict_inds -- список месяцев вперёд, для которых нужно сделать предсказание.
* generate_dataset -- возвращает обучающее множество для каждого классификатора
* generate_features -- генерирует примеры (итерируется по скважинам)
* create_features -- функция, внутри которой создаются фичи (только её и следуюет модифицировать, если Вы хотите добавить новые признаки)

In [12]:
def create_features(row):
    
    mean = row.mean().values
    last = row.iloc[-1].values
    sample = np.concatenate([mean, last])
    
    sample = pd.Series(sample)
    index = [name + '_mean' for name in row.columns] + \
              [name + '_last' for name in row.columns]
    sample.index = index
    
    return sample

In [13]:
def generate_features(row, min_lag, lag, predicts_ind, get='X', verbose=True):
    if min_lag <= 0 or predicts_ind <= 0 or min_lag > lag or min_lag + predicts_ind > row.shape[0] and get not in ['X', 'Y']:
        if verbose:
            print('Empty')
        return pd.Series()
    
    step = 0
    res = []
    index = []
    for step in range(step, step + row.shape[0] - lag - predicts_ind):
        data = row.iloc[step:step+lag]
        if data.shape[0] < min_lag and verbose:
            print('Added zeros in front')
            data = pd.concat([pd.DataFrame(np.zeros(lag - data.shape[0], data.shape[1])), data])
        if get == 'X':
            res.append(create_features(data))
            index.append(row.index[step][1])
        elif get == 'Y':
            res.append(row.iloc[step+lag+predicts_ind-1]['Нефть, м3'])
            index.append(row.index[step][1])
        
    return pd.DataFrame(res, index=index)

In [ ]:
def generate_dataset(data, min_lag, max_lag, predicts_inds, verbose):
    y = []
    x = []
    for i in predicts_inds:
        gf = lambda row, get: generate_features(row, min_lag, max_lag, i, get, verbose)
        tqdm.pandas(desc="Generate dataset (X-%i)" % i)
        x.append(data.groupby(level=0).progress_apply(lambda row: gf(row, 'X')))
        tqdm.pandas(desc="Generate dataset (Y-%i)" % i)
        y.append(data.groupby(level=0).progress_apply(lambda row: gf(row, 'Y')))
    return x, y

In [4]:
def train_valid_split(data, valid_size, n_split, min_lag=6, lag=12, predict_inds=[1,2,3,4,5,6,7,8,9,10,11,12], random_state=42, verbose=True):
    np.random.seed(random_state)
    wells = list(data.index.levels[0])
    train_x, train_y, valid_x, valid_y = [], [], [], []    
    for i in range(n_split):
        valid_wells = np.random.choice(wells, int(len(wells) * valid_size), replace=False)
        train_wells = list(set(wells).difference(valid_wells))
        if verbose:
            print('Train split is creating now')
            time.sleep(1)
        x, y = generate_dataset(data.loc[train_wells], min_lag, lag, predict_inds, verbose)
        train_x.append(x)
        train_y.append(y)
        if verbose:
            print('Valid split is creating now')
            time.sleep(1)
        x, y = generate_dataset(data.loc[valid_wells], min_lag, lag, predict_inds, verbose)
        valid_x.append(x)
        valid_y.append(y)
        
        if verbose:
            print('Splits were created: %i' % (i + 1))
            time.sleep(1)
    return train_x, train_y, valid_x, valid_y

In [ ]:
def generate_submit(row, lag):
    res = []
    
    data = row
    if row.shape[0] < lag:
        zeros = pd.DataFrame(np.zeros((lag - row.shape[0], row.shape[1])), columns=row.columns)
        data = pd.concat([zeros, row], ignore_index=True)
    res.append(create_features(data))
    
    return pd.DataFrame(res, index=[row.name])

In [11]:
train_x, train_y, valid_x, valid_y = train_valid_split(train_and_valid, 0.2, 5)

Train split is creating now


Generate dataset (Y-12): 100%|██████████| 186/186 [00:03<00:00, 54.00it/s]


Valid split is creating now


Generate dataset (Y-12): 100%|██████████| 46/46 [00:00<00:00, 51.57it/s]


Splits were created: 1
Train split is creating now


...

Splits were created: 4
Train split is creating now


Generate dataset (Y-12): 100%|██████████| 186/186 [00:03<00:00, 53.46it/s]


Valid split is creating now


Generate dataset (Y-12): 100%|██████████| 46/46 [00:00<00:00, 51.32it/s]


Splits were created: 5


Проводим валидацию:

In [16]:
err_sum = []
np.random.seed(42)
for split_ind in tqdm_notebook(range(len(train_x))):
    models = []
    for lag in range(12):
        rf = RandomForestRegressor(n_jobs=10)
        rf.fit(train_x[split_ind][lag], train_y[split_ind][lag])
        models.append(rf)
        
    err = np.array([])
    for model in models:
        y_target = valid_y[split_ind][lag].values.flatten()
        y = np.array(model.predict(valid_x[split_ind][lag]))
        
        err = np.concatenate([err, np.abs(y_target - y)])
    err_sum.append(np.sum(err) / err.shape[0])
    print(err_sum[-1])
print('Validation: %f' % np.mean(err_sum))

94.80124393131689
95.13702314078027
125.7932060001958
147.83956386591097
116.10421333558973
Validation: 115.935050


In [24]:
x, y = generate_dataset(train_and_valid, 5, 12, list(range(1, 13)), True)

Generate dataset (Y-12): 100%|██████████| 232/232 [00:04<00:00, 49.26it/s]


In [25]:
models = []
for lag in tqdm_notebook(range(12)):
    rf = RandomForestRegressor(n_jobs=10)
    rf.fit(x[lag], y[lag])
    models.append(rf)

In [26]:
submit_data = train_and_valid.loc[submit_wells].groupby(level=0).apply(lambda row: row.iloc[-12:]).groupby(level=0)
x_submit = submit_data.apply(lambda row: generate_submit(row, lag=12))

new_x = []
for row in order.iterrows():
    new_x.append(x_submit.loc[row[1]['well']].values[0])
x_submit = np.array(new_x)

In [27]:
res = []
for i in range(12):
    res.append(models[i].predict(x_submit))
    
df = pd.DataFrame(np.array(res).flatten(order='F'), columns=['Predict'])

df.to_csv('submit.csv')
df

,Predict
0,292.084
1,225.947
2,232.177
3,219.421
4,186.955
5,213.944
6,235.559
7,195.823
8,246.812
9,243.752
